<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>Introduction to the LSST data Butler</b> <br>
Contact author(s): Alex Drlica-Wagner, Melissa Graham <br>
Last verified to run: 2022-04-29 <br>
LSST Science Piplines version: Weekly 2021_49 <br>
Container Size: medium <br>
Targeted learning level: beginner <br>

**Description:** Learn about how to create a Butler and use it to query and access data.

**Skills:** Discover, query, retrieve, and display images and catalog data with the Generation 3 Butler.

**LSST Data Products:** Queries for calexp and coadd images.

**Packages:** lsst.daf.butler, lsst.geom, lsst.afw.coord

**Credit:** This tutorial was originally developed by Alex Drlica-Wagner in the context of the LSST Stack Club. Please consider acknowledging Alex Drlica-Wagner in any publications or software releases that make use of this notebook's contents.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-1.lsst.io">dp0-1.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

The Butler is the LSST Science Pipelines interface for managing, reading, and writing datasets. The Butler can be used to explore the contents of the DP0 data repository and access the DP0 data. The current version of the Butler (referred to as "Gen-3") is still under development, and this notebook may be modified in the future. Full Butler documentation can be found [here](https://pipelines.lsst.io/v/weekly/modules/lsst.daf.butler/index.html).

This notebook demonstrates how to:<br>
1. Create an instance of the Butler<br>
2. Explore the contents of the DP0 data repository<br>
3. Retrieve images using various query constraints<br>

### 1.1 Package Imports

Import general python packages and several packages from the LSST Science Pipelines, including the Butler package and AFW Display, which will be used to display images.
More details and techniques regarding image display with `AFW Display` can be found in the `rubin-dp0` GitHub Organization's [tutorial-notebooks](https://github.com/rubin-dp0/tutorial-notebooks) repository.

In [ ]:
# Generic python packages
import warnings
import numpy as np
import pylab as plt

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay
import lsst.geom as geom
import lsst.sphgeom
import lsst.afw.coord as afwCoord
afwDisplay.setDefaultBackend('matplotlib')

# Set a standard figure size to use
plt.rcParams['figure.figsize'] = (8.0, 8.0)

## 2. Create an instance of the Butler

To create the Butler, we need to provide it with a configuration for the data set (often referred to as a "data repository").
This configuration can be the path to a yaml file (often named `butler.yaml`), a directory path (in which case the Butler will look for a `butler.yaml` file in that directory), a shorthand repository label (i.e., `dp02`). If no configuration is passed, the Butler will use a default value (this is not recommended in most cases).

We can search for a list of known repository labels with:

In [ ]:
dafButler.Butler.get_known_repos()

Next we will construct a butler pointing to the dp02 repository.

In [ ]:
# Create an instance of the Butler pointing to the DP0.2 repository
config = 'dp02'
butler = dafButler.Butler(config)

# Note: This will trigger a warning from CFITSIO in w_2022_22.
# This warning can be safely ignored and will be corrected in the future.

More information on the Butler can be found through the help string of our Butler instance.

In [ ]:
# Learn more about the butler by uncommenting the following line.
#help(butler)

### 2. Explore the DP0 data repository

Butler repositories have both a database component and a file-like storage component. The database component can be accessed through the Butler registry, while file-like storage can be local (i.e., pointing to a directory on the local file system) or remote (i.e., pointing to cloud storage resources).
DP0 uses Simple Storage Service (S3) buckets, which are public cloud storage resources that are similar to file folders, store objects, and which consist of data and its descriptive metadata.

#### 2.1 Butler registry and collections

The database side of a data repository is called a `registry`.
The registry contains entries for all data products, and organizes them by _collections_, _dataset types_, and _data IDs_.
We can access a registry client directly as part of our Butler object:

In [ ]:
registry = butler.registry

In [ ]:
# Learn more about the registry by uncommenting the following line.
# help(registry)

Collections are lightweight groups of datasets such as self-consistent calibration datasets, the outputs of a processing run, and the set of all raw images for a particular instrument. 
Find more about collections [here](https://pipelines.lsst.io/v/weekly/modules/lsst.daf.butler/organizing.html#collections).

We can use the registry to investigate a repository by listing all collections. Below we list the first 10 collections as a demonstration.

In [ ]:
all_collections = sorted(registry.queryCollections())
for c in all_collections[:10]:
    print(c)
    
print('...')
print(f'Total number of collections: {len(all_collections)}')

That is a lot of collections! However, when you are accessing the DP0.2 data you will usually only care about one collection:

`2.2i/runs/DP0.2`

In the event that you are interested in exploring other collections, we include a little more information here that may help you parse out the naming conventions of various collections.

* `2.2i` - refers to the processing run of the LSST DESC DC2 data (the "i" stands for "imSim")
* `calib` - refers to calibration products that are used for instrument signature removal
* `runs` - refers to processed data products
* `refcats` - refers to the reference catalogs used for astrometric and photometric calibration
* `skymaps` - definitions for the _tract_ and _patch_ grids that coadds are built on
* `PREOPS-NNN` - collections that are linked to a specific pre-operations tickets
* `u` - the suggested convention for user-generated collections is `u/{username}/...`

If there is a specific "flavor" of collection that you are interested in, you can also query for collections using strings and wildcards.

In [ ]:
# List the collections containing the string 'calib'
calib_collections = sorted(registry.queryCollections('*calib*'))
for c in calib_collections:
    print(c)

<br>

Just to re-iterate, DP0 delegates can access the DP0.2 by selecting the `2.2i/runs/DP0.2` collection. Let's do this now by creating a new butler that specifies this collection.

In [ ]:
collection = "2.2i/runs/DP0.2"
butler = dafButler.Butler(config, collections=collection)
registry = butler.registry

#### 2.2 Butler DatasetType

The LSST Science Pipelines classify data products as `DatasetTypes`.
To demonstrate how to see the available `DatasetTypes`, the following cell prints them all to screen.

As individual `DatasetTypes` are defined globally and do not belong to a specific collection, the following query returns *all* that belong to the repository, not just in the collection of interest. 

In [ ]:
help(registry.queryDatasetTypes)

In [ ]:
all_dataset_types = sorted(registry.queryDatasetTypes())
for x in all_dataset_types[:10]:
    print(x)
    
print('...')
print(f'Total number of Dataset Types: {len(all_dataset_types)}')

<br>
Again, that is a lot of dataset types! Here we provide definitions of some of the most commonly used dataset types to help delegates understand the contents of the full DPO repository. 

* `calexp` - a single CCD of a processed visit image (PVI; individual calibrated exposures)
* `deepCoadd` - products related to the coadded (stacked) images, including catalogs of coadd sources
* `src` - a catalog of sources
* `skyMap` - geometric representations of the sky coverage
* `dia` - difference image analysis products

Again, it is possible to query for a specific substring of a dataset type.

In [ ]:
dia_dataset_types = sorted(registry.queryDatasetTypes('*object*'))
for x in dia_dataset_types:
    print(x)

In [ ]:
butler.get('src',dataId={'visit': 971990, 'detector': 1})

**Which data sets are most appropriate for DP0.2?** <br>
> Most DP0 delegates will only be interested in data sets with types `ExposureF` or `SourceCatalog`. 
> For images, stick to the `calexp` (processed visit images, or "PVIs") and `deepCoadd_calexp` (stacked PVIs).
> For catalogs, the `src` should be used with the `calexp` images, and the `deepCoadd_forced_src` are the most appropriate to be used with the `deepCoadd_calexp`.
> More information can be found in the DP0.2 Data Products Definitions Document (DPDD) at [dp0-2.lsst.io](http://dp0-2.lsst.io).

### 2.3 Butler data IDs

The data ID is a dictionary-like identifier for a data product.
Find more about the data IDs [here](https://pipelines.lsst.io/v/weekly/modules/lsst.daf.butler/dimensions.html#data-ids).

Each `DatasetType` uses a different set of keys in its data ID.
For example, in the `DatasetType` list printed to screen (above), next to `calexp` in curly brackets is listed the band, instrument, detector, physical_filter, visit_system, and visit.
These are the keys of the data ID for a `calexp`, which are also called "dimensions".

The data ID contains both *implied* and *required* keys.
For example, the value of *band* would be *implied* by the *visit*, because a single visit refers to a single exposure at a single pointing in a single band.

While it is possible to query for all available data without any constraints, such a query would take longer than we would like to spend in this notebook. However, we can use a partially specified dataId to select just the data products associated with a single visit (971990).

In the following cell, we query for `calexp` data associated with visit=971990 in our collection of interest. the full data IDs are printed to screen (for just a few examples).
Data IDs can be represented in code as regular Python `dict` objects, but when returned from the `Butler` the `DataCoordinate` class is used instead.
 
In the following cell, printing the data ID without specifying `.full` shows only the required keys.
The value of a single key, in this case *band*, can also be printed by specifying the key name.

The following cell will fail and return an error if the query is requesting a `DatasetRef` for data that does not exist.

In [ ]:
datasetRefs = registry.queryDatasets(datasetType='calexp',
                                     collections=collection, 
                                     dataId={'visit': 971990})

for i, ref in enumerate(datasetRefs):
    print(ref.dataId.full)
    print(ref.dataId)
    print(ref.dataId['band'])
    print(' ')
    if i > 2:
        break

The Uniform Resource Identifier (URI) is the closest thing to a "filepath to the data" that the Butler provides. 
Note that this URI does not refer to a local path on the filesystem.
There is no need to know exactly where the data live in order to access it - that's the power of the Butler!!

The following cell prints the URI to screen as a demonstration of an alternate way to uniquely identify data in the Butler.

In [ ]:
for i, ref in enumerate(datasetRefs):
    print('File URI: ', butler.getURI(ref))
    if i > 2:
        break

## 3. Querying data sets

#### 3.1 Butler queryDatasets

Above demonstrated a very simple use of `queryDatasets`, but additional query terms can also be used, such as band and visit.
When a query term is an equality, it can be specified like `band='g'`. 
When a query term is an inequality, it can be specified with `where`.
Details about Butler queries can be found [here](https://pipelines.lsst.io/v/weekly/modules/lsst.daf.butler/queries.html).


In [ ]:
datasetRefs = registry.queryDatasets(datasetType='calexp', band='g',
                                     where='visit > 900000 and visit < 910000',
                                     collections=collection)

for i, ref in enumerate(datasetRefs):
    print(ref.dataId.full)
    if i > 2:
        break

<br>

Each data ID key-value pair is associated with a metadata row called a `DimensionRecord`.
Like dataset types, these exist independent of any collection, but they are also identified by data IDs.

The `queryDimensionsRecords` method provides a way to query for these records.
Most of the arguments accepted by `queryDatasets` can be used here (including `where`).

An example of this is provided below:

In [ ]:
for dim in ['exposure', 'visit', 'detector']:
    print(list(registry.queryDimensionRecords(dim, where='visit = 971990 and detector=0'))[0])
    print()

Another query method, `queryDataIds`, can be used to query for data IDs independent of any dataset, but it's less useful for general data exploration.

It is also possible to pass `datasets` and `collections` to both `queryDataIds` and `queryDimensionRecords` in order to return records whose data IDs match those of existing datasets.
But this is quite a bit more subtle than searching directly for a dataset, and rarely wanted when exploring a data repository.

More information on all of the query methods can be found [here](https://pipelines.lsst.io/v/weekly/middleware/faq.html#when-should-i-use-each-of-the-query-methods-commands).

#### 3.2 Temporal queries

The following examples show how to query for data sets that include a desired coordinate and observation date.

Above, we can see that for visit 971990, the (RA,Dec) are (70.37770,-37.1757) and the observation date is 20251201.
But these are just human-readable summaries of the more precise spatial and temporal information stored in the registry, which are represented in Python by `Timespan` and `Region` objects, respectively.
`DimensionRecord` objects that represent spatial or temporal concepts (a `visit` is both) have these objects attached to them.

Retrieve the `DimensionRecord` for a visit and show its timespan and region.

In [ ]:
(record,) = registry.queryDimensionRecords('visit', visit=971990)

print(record.timespan)
print(' ')
print(record.region)

If the timespan or spatial region that are being used as query constraints are already associated with a data ID in the database, the spatial and temporal overlap constraints are automatic.
For example, if we query for `deepCoadd` datasets with a `visit`+`detector` data ID, we'll get just the ones that overlap that observation and have the same band (because a visit implies a band):

In [ ]:
for ref in registry.queryDatasets("deepCoadd", visit=971990, detector=50):
    print(ref)

To query for dimension records or datasets that overlap an arbitrary time range, we can use the `bind` argument to pass times through to `where`.
Using `bind` to define an alias for a variable saves us from having to string-format the times into the `where` expression.
Note that a `dafButler.Timespan` will accept a `begin` or `end` value that is equal to `None` if it is unbounded on that side.

Use `bind` and `where`, along with [astropy.time](https://docs.astropy.org/en/stable/time/index.html), to look for visits within one minute of this one on either side.

In [ ]:
# import astropy.time
# minute = astropy.time.TimeDelta(60, format="sec")
# timespan = dafButler.Timespan(record.timespan.begin - minute, record.timespan.end + minute)

# for visit in registry.queryDimensionRecords("visit", where="visit.timespan OVERLAPS my_timespan", 
#                                             bind={"my_timespan": timespan}):
#     print(visit.id, visit.timespan, visit.physical_filter)

In [ ]:
import astropy.time
minute = astropy.time.TimeDelta(60, format="sec")
timespan = dafButler.Timespan(record.timespan.begin - minute,
                              record.timespan.end + minute)

datasetRefs = registry.queryDatasets("calexp", 
                                     where="visit.timespan OVERLAPS my_timespan",
                                     bind={"my_timespan": timespan})

for i, ref in enumerate(datasetRefs):
    print(ref)
    if i > 6:
        break

#### 3.3 Spatial queries

Arbitrary spatial queries are not supported at this time, such as the "POINT() IN (REGION)" example found in this [Butler queries](https://pipelines.lsst.io/v/weekly/modules/lsst.daf.butler/queries.html) documentation.
In other words, at this time it is only possible to do queries involving regions that are already "in" the data repository, either because they are HTM pixel regions or because they are tract/patch/visit/visit+detector regions.

Thus, for this example we use the set of dimensions that correspond to different levels of the HTM (hierarchical triangular mesh) pixelization of the sky ([HTM primer](http://www.skyserver.org/htm/)).
The process is to transform a region or point into one or more HTM identifiers (HTM IDs), and then create a query using the HTM ID as the spatial data ID.
The `lsst.sphgeom` library supports region objects and HTM pixelization in the LSST Science Pipelines.

Import the `lsst.sphgeom` package, initialize a sky pixelization to level 10 (the level at which one sky pixel is about five arcmin across), and find the HTM ID for a desired sky coordinate.

In [ ]:
pixelization = lsst.sphgeom.HtmPixelization(10)

In [ ]:
htm_id = pixelization.index(
    lsst.sphgeom.UnitVector3d(
        lsst.sphgeom.LonLat.fromDegrees(70.376995, -37.175736)
    )
)

# Obtain and print the scale to provide a sense of the size of the
# sky pixelization being used
circle = pixelization.triangle(htm_id).getBoundingCircle()
scale = circle.getOpeningAngle().asDegrees()*3600.
level = pixelization.getLevel()
print(f'HTM ID={htm_id} at level={level} is a ~{scale:0.2}" triangle.')

In [ ]:
datasetRefs = registry.queryDatasets("calexp", htm20=htm_id,
                                     where="visit.timespan OVERLAPS my_timespan",
                                     bind={"my_timespan": timespan})

for i, ref in enumerate(datasetRefs):
    print(ref)
    if i > 6:
        break

Thus, with the above query, we have uniquely identified the visit and detector for our desired temporal and spatial constraints.

Note that if a smaller HTM level is used (like 7), which is a larger sky pixel (~2200 arcseconds), the above query will return many more visits and detectors which overlap with that larger region. Try it and see!

Note that queries using the HTM ID can also be used to, e.g., find the set of all i-band `src` catalog data products that overlap this point.

In [ ]:
for i, src_ref in enumerate(registry.queryDatasets("src", htm20=htm_id,
                                                   band="i")):
    print(src_ref)
    if i > 2:
        break

Why is does that search take tens of seconds?
The butler's spatial reasoning is designed to work well for regions the size of full data products, like detector- or patch-level images and catalogs, and it's a poor choice for object-scale searches.
The above search is slow in part because `queryDatasets` searches for all `src` datasets that overlap a larger region and then filters the results down to the specified HTM ID pixel.

For searches of these scales, it is often more efficient to use the TAP service, as demonstrated in later tutorials.